In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
import predict
import base

In [2]:
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取休假表
calenders = base.ReadCalendarCSV();

In [4]:
predict.initTrainArimasData('test', shop_open_dates, user_pay_counts, pd.to_datetime('2016-10-10'), pd.to_datetime('2016-10-17'))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
#执行ArimaTrain.R

In [5]:
arima_df = predict.predictArimas('test')

In [6]:
#ARIMA 时间序列预测
evaluations = {}


for id in range(1,2001):
    y_arima = arima_df.ix[id].values
    y_Test = base.getTestCount(user_pay_counts, id, pd.to_datetime('2016-10-18'), pd.to_datetime('2016-10-31')).values
    e = base.evaluation_tm(y_Test, y_arima)
    print("id=%d evaluation=%f"%(id, e))
    evaluations[id] = e
    
arima_evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
arima_evaluation_df.columns = ['evaluation']
arima_evaluation_df.index.name = 'id'
arima_evaluation_df.sort_values('evaluation')


id=1 evaluation=0.065055
id=2 evaluation=nan
id=3 evaluation=0.129179
id=4 evaluation=0.180767
id=5 evaluation=0.223187
id=6 evaluation=0.060158
id=7 evaluation=0.109939
id=8 evaluation=0.109929
id=9 evaluation=0.063390
id=10 evaluation=0.076233
id=11 evaluation=0.080677
id=12 evaluation=0.119970
id=13 evaluation=0.031103
id=14 evaluation=0.067779
id=15 evaluation=0.082247
id=16 evaluation=0.087476
id=17 evaluation=0.061765
id=18 evaluation=0.225546
id=19 evaluation=0.110196
id=20 evaluation=0.048154
id=21 evaluation=0.108485
id=22 evaluation=0.080201
id=23 evaluation=0.474952
id=24 evaluation=0.177759
id=25 evaluation=0.062969
id=26 evaluation=0.206718
id=27 evaluation=0.063259
id=28 evaluation=0.107322
id=29 evaluation=0.144746
id=30 evaluation=0.095052
id=31 evaluation=0.139904
id=32 evaluation=0.209434
id=33 evaluation=0.118100
id=34 evaluation=0.081933
id=35 evaluation=0.148027
id=36 evaluation=0.079828
id=37 evaluation=0.079621
id=38 evaluation=0.142819
id=39 evaluation=0.136894


In [8]:
arima_evaluation_df['evaluation'].mean()

0.11677896622383281

In [7]:
arima_evaluation_df[arima_evaluation_df['evaluation'].isnull()]


,evaluation
id,
2,NaN
191,NaN
632,NaN
659,NaN
700,NaN
810,NaN
1243,NaN
1459,NaN
1462,NaN


In [9]:
arima_evaluation_df = arima_evaluation_df.fillna(1)
arima_evaluation_df.to_csv("./data/dataset/dataset/arima_evaluation.txt", sep='\t', index=True, encoding='UTF-8')

In [ ]:
#0.11460046737813145

In [ ]:
def initArimaData(id):
    open_time = shop_open_dates.loc[id]['date'];
    if open_time < pd.to_datetime('2015-7-1'):
        open_time = pd.to_datetime('2015-7-1')
    df = base.countShopPayTimePeriods(user_pay_counts, id, date_range=[open_time, pd.to_datetime('2016-10-17')], time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
    
    return df

In [ ]:
arima_df = initArimaData(id)
arima_df.to_csv("./data/dataset/dataset/arima/" + str(id), index=False, encoding='UTF-8')

In [ ]:
arima_df = pd.read_csv("./data/dataset/dataset/arima/" + str(id) + "_result", encoding='UTF-8', index_col=0)
arima_df['x'] = arima_df['x'].astype(np.int32)

In [ ]:
y_arima = arima_df['x'].values
e = base.evaluation(y_Test, y_arima)
print("id=%d evaluation=%f"%(id, e))
print(y_Test)
print(y_arima)
PlotPredAndTestPayCount(y_Test, y_arima, train_mean)

In [ ]:
for id in range(1, 2001):
    arima_df = initArimaData(id)
    arima_df.to_csv("./data/dataset/dataset/arima/" + str(id), index=False, encoding='UTF-8')

In [ ]:
#ARIMA 时间序列预测
evaluations = {}
arima_df = base.ReadArimaPayCountCSV()

for id in range(1,2001):
    y_arima = arima_df.ix[id].values
    y_Test = base.getTestCount(user_pay_counts, id, pd.to_datetime('2016-10-18'), pd.to_datetime('2016-10-31')).values
    e = base.evaluation_tm(y_Test, y_arima)
    print("id=%d evaluation=%f"%(id, e))
    evaluations[id] = e
    
arima_evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
arima_evaluation_df.columns = ['evaluation']
arima_evaluation_df.index.name = 'id'
arima_evaluation_df.sort_values('evaluation')
arima_evaluation_df.to_csv("./data/dataset/dataset/arima_evaluation.txt", sep='\t', index=True, encoding='UTF-8')